<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="Defining the environment and choices made during the project" data-toc-modified-id="Defining the environment and choices made during the project-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Defining the environment and choices made during the project</a></span></li><li><span><a href="#Global Navigation" data-toc-modified-id="Global Navigation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Global Navigation</a></span></li><li><span><a href="#Filtering" data-toc-modified-id="Filtering-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Filtering</a></span></li><li><span><a href="#Vision" data-toc-modified-id="Vision-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Vision</a></span></li><li><span><a href="#Local avoidance" data-toc-modified-id="Local avoidance-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Local avoidance</a></span></li><li><span><a href="#Motion control" data-toc-modified-id="Motion control-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Motion control</a>
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

# Introduction

In the scope of our course, "Basics of Mobile Robotics," we undertook a comprehensive project that integrated the key concepts covered throughout the semester's exercise sessions. Our objective was to synthesize our understanding of Global Navigation, Local Navigation, Filtering, Motion Control, and Vision into a practical application. The project encompassed several essential goals:
- Environment Creation: We created a setting with a predetermined set of obstacles. The Thymio robot has to find its way around this environment on its own, avoiding obstacles without using sensors to detect them.
- Pathfinding: The main goal was to direct the Thymio robot to a destination located anywhere in the surroundings from an arbitrary starting place. We were able to evaluate the system’s adaptability because of its capacity to change the target’s location
- Motion control and pose estimation: Precise control was applied to enable the robot to travel along the predetermined path. Kalman filtering was used to estimate the pose accurately, which was necessary to do this.
- Obstacle Avoidance: The Thymio was able to avoid obstacles that were purposefully positioned in its route at any given time by using the front proximity sensors. 

This project helped us gain a greater understanding of the principles of mobile robotics by not only providing a comprehensive application of theoretical concepts but also challenging us to integrate and apply these principles in a real-world robotic scenario.



# Defining the environment and choices made during the project


In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

# Gloval Navigation


In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

# Filtering


In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

# Vision


In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

In [ ]:
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH
HHHHHHHHHHHHHHHHHHHHHHHH

# Local avoidance


Our main goal in the local navigation part is to make it possible for the Thymio robot to move dynamically through its environment. As the robot follows its global path, it frequently encounters physical obstacles randomly introduced into its trajectory. We implemented a function called test_saw_obstacle to determine whether the Thymio has recognized an impediment based on readings from its proximity sensors in order to overcome this difficulty. If an obstacle is found, this function returns True; if not, it returns False.


In [ ]:
def test_saw_obstacle(threshold, verbose=True):
    
    if any([x > threshold for x in node['prox.horizontal'][0:5]]):
        if verbose: print("\t\t Obstacle detected")
        return True
    
    return False

Afterward, using asynchronous programming, an asynchronous function called local_avoidance was developed to implement a local obstacle avoidance behavior for the robot. Five horizontal proximity sensor readings were used to determine the speed adjustments for each motor, which were represented by an array called obstacleSpeedGain that was initialized within this function. The sensor readings are multiplied by the corresponding gain values to calculate the adjustments. Using the test_saw_obstacle function and a given threshold, the program does an obstacle detection check within the loop. If an obstacle is detected, the system engages in a local avoidance behavior, which involves adjusting motor speeds (speedL and speedR) based on obstacle readings and the defined obstacleSpeedGain. Finally, node.set_variables(motors(speedL, speedR)) is used to change the motor speeds. 

In [ ]:
async def local_avoidance(motor_speed=100, threshold=0, verbose=False):
    obstacleSpeedGain = [6, 4, -2, -6, -8]
    saw_obstacle = True
    speedL = motor_speed
    speedR = motor_speed
    prev_state = "global"

    if verbose:
        print("\t Moving forward")

    while saw_obstacle:
        if test_saw_obstacle(threshold, verbose=False):
            if prev_state == "global":
                prev_state = "local"
                print("\t Transitioning to local avoidance")

            for i in range(5):
                speedL += node["prox.horizontal"][i] * obstacleSpeedGain[i] // 100
                speedR += node["prox.horizontal"][i] * obstacleSpeedGain[4 - i] // 100

          
            await node.set_variables(motors(speedL, speedR))
            if verbose:
                print("\t\t Adjusting for obstacle")

        else:
            if prev_state == "local":
                #prev_state = "global"
                print(prev_state)
                print("\t Transitioning to global navigation")
                
               

            # Reset motor speeds when no obstacles are detected
            speedL = motor_speed
            speedR = motor_speed
            await node.set_variables(motors(speedL, speedR))
            
            # Add a small delay to avoid continuously checking for obstacles too quickly
            await client.sleep(0.1)




# Motion control
